# Preprocessing

### Préparation des colonnes pré existantes

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import kurtosis
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import torch


# ---------------------------
# CONFIG
# ---------------------------
file_path = "pf_2020-03-30_filtered_downsampled.csv"

time_col = "time"        # nom de la colonne datetime dans le csv
amp_col = "amplitude"           # si None, automatiquement détecté ou calculé
fs = 1/60                   # 1/60 Hz (1 minute)
win_min = 10         # window pour features (10 minutes)
step_minuts = 10        # step (ici non chevauché). mettre < win_min pour chevauchement
env_window = 200         # smoothing median window (200 valeurs)
min_rows_for_env = env_window
seq_length = 60       # longueur des séquences pour le modèle (en minutes)
batch_size = 64
device = "cuda" if torch.cuda.is_available() else "cpu"

# Classe mapping (modifiable)
# Par défaut : classes (0..4) :
# 0 => delay > 24h
# 1 => 16h < delay <= 24h
# 2 => 1h < delay <= 12h
# 3 => 0h < delay <= 1h
# 4 => delay <= 0h (en cours)
# NOTE: ceci est paramétrable ci-dessous
thresholds_hours = {
    "0": 0,
    "1": 1,
    "2": 12,
    "3": 16,
    "4": 24
}

# ---------------------------
# 1. Lecture, homogénéisation titre colonnes et colonnes temporelles
# ---------------------------
df = pd.read_csv(file_path)
df.columns = [c.strip().lower() for c in df.columns]
# ---------------------------
# Division de la colonne "time"
# ---------------------------
df[time_col] = pd.to_datetime(df[time_col], errors="coerce")
if df[time_col].isna().any():
    raise ValueError("Des valeurs non parsables dans la colonne time. Vérifier le format.")

# ajouter year/month/day/hour pour le modèle (toujours numériques)
df["year"]  = df[time_col].dt.year.astype(np.int16)
df["month"] = df[time_col].dt.month.astype(np.int8)
df["day"]   = df[time_col].dt.day.astype(np.int8)
df["hour"]  = df[time_col].dt.hour.astype(np.int8)
df["minute"]= df[time_col].dt.minute.astype(np.int8)
df["seconde"]= df[time_col].dt.second.astype(np.int8)


start = pd.to_datetime('2020-04-02T08:20:00.000000Z') #Date de début de l'éruption
end = pd.to_datetime('2020-04-06T09:30:00.000000Z') #Date de fin de l'éruption

default_case = pd.NaT

conditions = [
    (df[time_col] > end),                           # 1. APRÈS l'intervalle
    (df[time_col] >= start) & (df[time_col] <= end) # 2. PENDANT l'intervalle
]
choices = [
    pd.NaT,          # 1. Si APRÈS -> NaN
    pd.Timedelta(0)  # 2. Si PENDANT -> 0
]

df['delai_eruption'] = np.select(conditions, choices, default=pd.NaT)


# suppression de la colonne time suite à sa division
df.drop("time", axis = 1)

# ---------------------------
#Extraction du type de composante : horizontale ou verticale
# ---------------------------
def type_component(channel):
    c = str(channel).upper()

    # EHZ, SHZ, BHZ, HHZ → verticale
    if c.endswith("Z"):
        return "vertical"

    # HHE, HHN, EHE, etc → horizontale
    return "horizontal"

df["component_type"] = df["channel"].apply(type_component)

# Encodage numérique pour le modèle
df["component_flag"] = df["component_type"].map({
    "horizontal": 0,
    "vertical": 1
})




### Création et ajout des nouvelles features dans le dataframe

In [14]:
# ---------------------------
# 3. Fonctions features
# ---------------------------
def shannon_entropy(segment, bins=50):
    p, _ = np.histogram(segment, bins=bins, density=True)
    p = p[p > 0]
    if p.size == 0:
        return 0.0
    return -np.sum(p * np.log2(p))

def frequency_index_proxy(segment):
    # proxy amplitude-based for fs=1Hz
    med = np.median(np.abs(segment))
    high = segment[np.abs(segment) > med]
    low  = segment[np.abs(segment) <= med]
    E_high = np.sum(high**2)
    E_low  = np.sum(low**2)
    if E_low == 0:
        return np.nan
    return float(E_high) / float(E_low)

# ---------------------------
# 4. Sliding windows: calcul des features
#    WARNING: sur des millions de lignes, cette boucle est lente.
#    Pour des datasets massifs, vectoriser ou utiliser numba/parallel est recommandé.
# ---------------------------
signal = df[amp_col].values.astype(float)
n = len(signal)
win = int(win_min)             # ex 60
step = int(step_minuts)
indices = range(0, n - win + 1, step)

feat_list = []
times_out = []

for i in indices:
    seg = signal[i:i+win]
    t_center = df[time_col].iloc[i + win - 1]  # horodatage de fin de fenêtre (alignement comme étude)
    SE = shannon_entropy(seg)
    K  = float(kurtosis(seg, fisher=True, bias=False))
    FI = frequency_index_proxy(seg)
    std = float(np.std(seg))
    mean = float(np.mean(seg))
    med = float(np.median(seg))
    per90 = float(np.percentile(seg,90))
    per10 = float(np.percentile(seg,10))
    tension = per90 - per10

    feat_list.append((t_center, SE, K, FI, std, mean, med, per90, per10, tension))
    times_out.append(t_center)

# DataFrame features
df_feat = pd.DataFrame(feat_list, columns=[
    "time", "SE","Kurtosis","FI","std","mean","median","per90","per10","tension"
])

df_feat = df_feat.set_index("time")

# conserver la colonne time dans le dataset final (sous forme datetime index + une colonne time si nécessaire)
df_feat[time_col] = df_feat.index

# ---------------------------
# 5. Enveloppe median smoothing (200 valeurs)
# ---------------------------
if len(df_feat) < env_window:
    # option : utiliser min_periods=1 pour avoir valeurs même si < env_window
    df_feat["SE_env"] = df_feat["SE"].rolling(env_window, min_periods=1).median()
    df_feat["FI_env"] = df_feat["FI"].rolling(env_window, min_periods=1).median()
    df_feat["Kurt_env"] = df_feat["Kurtosis"].rolling(env_window, min_periods=1).median()
    # idem pour autres stats si souhaité
    df_feat["std_env"] = df_feat["std"].rolling(env_window, min_periods=1).median()
else:
    df_feat["SE_env"] = df_feat["SE"].rolling(env_window).median()
    df_feat["FI_env"] = df_feat["FI"].rolling(env_window).median()
    df_feat["Kurt_env"] = df_feat["Kurtosis"].rolling(env_window).median()
    df_feat["std_env"] = df_feat["std"].rolling(env_window).median()

# ---------------------------
# 6. Joindre colonnes date/year/month/day/hour depuis la table d'origine
#    (pour que modèle ait ces features temporelles)
# ---------------------------
# on fait un merge left-on time (index) pour récupérer year/month/day/hour
orig_time_df = df[[time_col,"year","month","day","hour","minute"]].set_index(time_col)
# aligner via nearest join with tolerance = step_seconds (facultatif)
df_feat = df_feat.join(orig_time_df, how="left")

# ---------------------------
# 7. Création du label à partir de 'delai_eruption' si présente dans df original
#    else lever erreur ou construire via table fournie
# ---------------------------
# On accepte différents formats : Timedelta / seconds / hours / NaN

# Si df original contient 'delai_eruption' (Timedelta) aligned on original time,
# il faut produire une colonne delai_eruption pour df_feat.
# Approche : si original df contient 'delai_eruption', faire forward fill / resample:
if "delai_eruption" in df.columns:
    # aligner : prendre la valeur du delai à la fin de la fenêtre
    # index df_feat is times at window end -> map from original df by exact match (may fail)
    # faire nearest merge:
    df_temp = df[[time_col,"delai_eruption"]].set_index(time_col)
    df_feat = df_feat.join(df_temp, how="left")
    # si delai_eruption est Timedelta, le garder ; sinon convertir en secondes si numeric
else:
    raise ValueError("Aucune colonne 'delai_eruption' trouvée dans le CSV d'origine. Fournir la table d'éruptions ou la colonne delai_eruption.")

# Normaliser le format de delai_eruption : convertir tout en float seconds
def delai_to_hours(x):
    if pd.isna(x):
        return np.nan
    # Timedelta
    if isinstance(x, pd.Timedelta):
        return x.total_seconds() / 3600.0
    # numpy timedelta64
    if np.issubdtype(type(x), np.timedelta64):
        return pd.to_timedelta(x).total_seconds() / 3600.0
    # string parsable as timedelta? try to parse numbers
    if isinstance(x, str):
        try:
            # tenter parse ISO duration or "HH:MM:SS" etc.
            td = pd.to_timedelta(x)
            return td.total_seconds() / 3600.0
        except Exception:
            try:
                return float(x) / 3600.0   # si x donné en secondes
            except:
                return np.nan
    # numeric seconds or hours
    if isinstance(x, (int, float, np.floating, np.integer)):
        # supposer en secondes si valeur > 24*3600 sinon si déjà en heures? difficile => supposer secondes
        # si < 1000 on considère c'est déjà en heures
        if abs(x) > 1000:
            return float(x) / 3600.0
        else:
            return float(x)
    return np.nan

df_feat["delai_hours"] = df_feat["delai_eruption"].apply(delai_to_hours)

# ---------------------------
# 8. Mapping labels (paramétrable)
# ---------------------------
def label_from_delay_hours(h):
    # h: float hours or nan
    if pd.isna(h):
        return 0   # class 0 = "pas de risque (>24h)" par défaut pour NaN
    if h <= 0:
        return 4   # en cours
    if h <= 1:
        return 3   # <1h
    if h <= 12:
        return 2   # <12h
    if h <= 16:
        return 1   # >16h class (intermédiaire)
    # else h > 16 -> class 0 (>24h / no risk) ; ajuster si besoin
    return 0

df_feat["label"] = df_feat["delai_hours"].apply(label_from_delay_hours).astype(np.int64)


### Encodage

In [ ]:
# ---------------------------
# 9. Features finales et encodage
# ---------------------------
# Colonnes numériques à garder
numeric_features = ["SE","Kurtosis","FI","std","mean","median","per90","per10","tension",
                    "SE_env","FI_env","Kurt_env","std_env",
                    "year","month","day","hour","minute"]

# garder uniquement celles existantes
numeric_features = [c for c in numeric_features if c in df_feat.columns]

# Colonnes catégorielles à encoder (ex: station, channel, component)
categorical_candidates = [c for c in df.columns if c.lower() in ("station","channel","network")]
categorical_candidates = [c for c in categorical_candidates if c in df.columns]
# également si present une colonne 'component' ou 'channel'
if "channel" in df.columns:
    categorical_candidates.append("channel")
categorical_candidates = list(dict.fromkeys(categorical_candidates))

# Build preprocess pipeline
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_candidates)
    ],
    remainder="drop"
)

# Remplir NaN numériques par median avant scaler
df_feat[numeric_features] = df_feat[numeric_features].fillna(df_feat[numeric_features].median())

# Remplir catégoriques na par 'unk'
for c in categorical_candidates:
    df[c] = df[c].fillna("unk")
    # aligner au df_feat times: map nearest value from original df if needed
    # simple approach: forward fill in original df then join earlier would have done it.

# Fit transformer on entire dataset (ou training only, best practice: fit on train)
X_num = df_feat[numeric_features].values
X_cat = df.loc[df_feat.index, categorical_candidates].values if len(categorical_candidates)>0 else None

if len(categorical_candidates)>0:
    X_cat = df[categorical_candidates].loc[df_feat.index].fillna("unk").values
    X_combined = preprocessor.fit_transform(pd.concat([df_feat[numeric_features], df[categorical_candidates].loc[df_feat.index]], axis=1))
else:
    X_combined = numeric_transformer.fit_transform(X_num)

# final feature names count
n_features = X_combined.shape[1]


### Création des séquences pour le modèle transformer et subdivision en train/test/val

In [ ]:
# ---------------------------
# 10. Construction des sequences (sliding) pour le modèle
# ---------------------------
# On construit sequences non chevauchantes (ou chevauchantes selon step_sequence)
step_seq = 1  # si 1 -> séquences glissantes à chaque point; si seq_length -> non chevauchées
X = X_combined
y = df_feat["label"].values
T = len(X)

seqs = []
labels = []
times_seq = []

for i in range(0, T - seq_length, step_seq):
    seq = X[i:i+seq_length]
    # label associé au temps de fin de séquence (alignement)
    lab = y[i+seq_length-1]
    seqs.append(seq)
    labels.append(lab)
    times_seq.append(df_feat.index[i+seq_length-1])

X_seq = np.stack(seqs)            # shape (N_seq, seq_length, n_features)
y_seq = np.array(labels)          # shape (N_seq,)

# ---------------------------
# 11. Train/Val/Test split temporel
# ---------------------------
N = len(X_seq)
train_frac = 0.7
val_frac = 0.15
test_frac = 0.15

i_train_end = int(N * train_frac)
i_val_end   = int(N * (train_frac + val_frac))

X_train = X_seq[:i_train_end]
y_train = y_seq[:i_train_end]
X_val   = X_seq[i_train_end:i_val_end]
y_val   = y_seq[i_train_end:i_val_end]
X_test  = X_seq[i_val_end:]
y_test  = y_seq[i_val_end:]

# Convert to torch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_t = torch.tensor(y_train, dtype=torch.long).to(device)
X_val_t   = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_t   = torch.tensor(y_val, dtype=torch.long).to(device)
X_test_t  = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_t  = torch.tensor(y_test, dtype=torch.long).to(device)

train_ds = TensorDataset(X_train_t, y_train_t)
val_ds   = TensorDataset(X_val_t, y_val_t)
test_ds  = TensorDataset(X_test_t, y_test_t)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False)